In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
import itertools
import matplotlib.pyplot as plt
from  sklearn.metrics import mean_squared_error as mse
from astropy.io import fits
import matplotlib.pyplot as plt
import os, sys
from numba import njit, jit
from math import isnan
from sklearn.preprocessing import MinMaxScaler

In [15]:
folder_lc = "/work/work_teamEXOPLANET/KOI_LC/"
#Clean Light Curves
lc_kepler = np.load(folder_lc+"npy/KOI_LC_init.npy")
lc_kepler_times = np.load(folder_lc+'npy/KOI_LC_time.npy')

In [16]:
total_time_list = []
for lc,times in zip(lc_kepler,lc_kepler_times):
    time_list = []
    aux_t = times[0]
    for i, data in enumerate(lc):
        if not isnan(data):
            time_list.append(times[i] - aux_t)
        else:
            time_list.append(np.nan)
            #time_list.append(0)
            if (times[i] != times[-1]):
                aux_t = times[i+1]
    total_time_list.append(time_list)

In [ ]:
indice = 14
plt.figure(figsize=(30,10))
plt.plot(total_time_list[indice],'bo')
plt.show()

## Valores entre -1 y 1

In [18]:
## no python: njit -- no soporta lista de lista, lista de arrays, si soporta: lista de sets

########### no lo logre hacer funcionar
#@njit(parallel=True, cache=False, fastmath=True)
def extract_nans(fluxs, plot=True):
    ###extrae los nans y deja tramos de valores continuos
    lc_wind_nan = []
    lengths = []

    sublist = [] #para que sepa que el tipo es float
    for value in fluxs:
        if np.isnan(value) and len(sublist) != 0:
            if len(sublist) != 1: ##borrar tramos de largo 1??
                lc_wind_nan.append(np.asarray(sublist))
                lengths.append(len(sublist))
            sublist = [] #para que sepa que el tipo es float
        elif np.isnan(value) and len(sublist) == 0:
            continue
        else: #if value not nan
            sublist.append(value) 

    if plot:
        lengths = np.asarray(lengths)   
        #print("Cantidad de tramos: ",len(lengths))
        #print("Largo promedio de tramos: ", np.mean(lengths))
        #print("Mediana de largo de tramos: ", np.median(lengths))
        #print("Min de largo de tramos: ", np.min(lengths))
        #print("Max de largo de tramos: ", np.max(lengths))
    return lc_wind_nan


def prepare_lc(fluxs): #dividir por max
    fluxs = np.asarray(fluxs)
    fluxs = fluxs.reshape(-1,1)
    scaler = MinMaxScaler((-1, 1))
    scaler.fit(fluxs)
    fluxs = scaler.transform(fluxs)
    fluxs = fluxs.ravel()
    return fluxs#fluxs/np.abs(np.nanmin(fluxs))

@njit(parallel=True, cache=False, fastmath=True)
def det_state(a, b, n_sta):
    topes=np.linspace(a, b, n_sta+1)
    estados= []
    ind=1
    for top in topes[:-1]:
        estados.append((top,topes[ind]))
        ind+=1
    return estados

@njit(parallel=False, cache=True, fastmath=True)
def det_state_2ways(a, b, n_sta_up, n_sta_low):
    estados_up = det_state(a, 0, n_sta_up)
    estados_low = det_state(0, b, n_sta_low)
    return estados_up + estados_low

@njit(parallel=False, cache=True, fastmath=True)
def det_celda(num, estados): 
    for celda, (est_low, est_up) in enumerate(estados):
        if num<est_low and num>=est_up:
            return celda
    return 0 #if sale por arriba (mayor a 1)

@njit(parallel=False, cache=True, fastmath=True)  #no se pudo paralelizar por el acceso a lista "ind -1"
def add_transitions(fluxs, transition_m, states):
    for ind in range(1,len(fluxs)):
        init_s = det_celda(fluxs[ind-1], states)
        fin_s = det_celda(fluxs[ind], states)
        transition_m[init_s, fin_s] += 1
    return transition_m

@njit(parallel=True, cache=False, fastmath=True)
def manual_HMM(wind_fluxs, n_sta_up, n_sta_low=0):
    if n_sta_low == 0: 
        n_sta_low = n_sta_up 
    n_sta = n_sta_up+n_sta_low
    
    transition_m = np.zeros((n_sta,n_sta))
    
    states = det_state_2ways(1,-1, n_sta_up=n_sta_up, n_sta_low=n_sta_low) 
    for fluxs in wind_fluxs:
        add_transitions(fluxs, transition_m, states)
    
    for i in range(n_sta):
        suma_i = 0
        for j in range(n_sta):
            transition_m[i,j] +=1 #priors
            suma_i += transition_m[i,j]
        transition_m[i] = transition_m[i]/suma_i #normalize
    return transition_m


In [20]:
%%time
listita = [[10,10],[10,15],[10,20],[10,30],[15,10],[15,15],[15,20],[15,30],[5,10],[5,15],[5,20],[5,30]]
for values in listita:
    final_npy = []
    i=0
    for lc_our_detrend in total_time_list:   
        print ("recuperando curva",i+1)
        fluxs = prepare_lc(lc_our_detrend) #divide by min
        #plt.figure(figsize=(15,10))
        #plt.plot(fluxs)
        #plt.show()
        lc_tramos =  extract_nans(fluxs) #extract nans  
        transition_m = manual_HMM(lc_tramos, n_sta_up=values[0], n_sta_low=values[1]) #si se dejan 5 arriba y 10 abajo cambia harto...

        i+=1
        #plt.imshow(transition_m, cmap='Blues', vmin=0, vmax=1) #el plot se ve raro por esto..
        #plt.show()
        final_npy.append(transition_m)
        
    final_npy = np.asarray(final_npy)
    np.save('/work/work_teamEXOPLANET/MTF_gabo/npys/MTF_time_u%s_d%s.npy'%(str(values[0]),str(values[1])), final_npy)

recuperando curva 1
recuperando curva 2
recuperando curva 3
recuperando curva 4
recuperando curva 5
recuperando curva 6
recuperando curva 7
recuperando curva 8
recuperando curva 9
recuperando curva 10
recuperando curva 11
recuperando curva 12
recuperando curva 13
recuperando curva 14
recuperando curva 15
recuperando curva 16
recuperando curva 17
recuperando curva 18
recuperando curva 19
recuperando curva 20
recuperando curva 21
recuperando curva 22
recuperando curva 23
recuperando curva 24
recuperando curva 25
recuperando curva 26
recuperando curva 27
recuperando curva 28
recuperando curva 29
recuperando curva 30
recuperando curva 31
recuperando curva 32
recuperando curva 33
recuperando curva 34
recuperando curva 35
recuperando curva 36
recuperando curva 37
recuperando curva 38
recuperando curva 39
recuperando curva 40
recuperando curva 41
recuperando curva 42
recuperando curva 43
recuperando curva 44
recuperando curva 45
recuperando curva 46
recuperando curva 47
recuperando curva 48
r

recuperando curva 379
recuperando curva 380
recuperando curva 381
recuperando curva 382
recuperando curva 383
recuperando curva 384
recuperando curva 385
recuperando curva 386
recuperando curva 387
recuperando curva 388
recuperando curva 389
recuperando curva 390
recuperando curva 391
recuperando curva 392
recuperando curva 393
recuperando curva 394
recuperando curva 395
recuperando curva 396
recuperando curva 397
recuperando curva 398
recuperando curva 399
recuperando curva 400
recuperando curva 401
recuperando curva 402
recuperando curva 403
recuperando curva 404
recuperando curva 405
recuperando curva 406
recuperando curva 407
recuperando curva 408
recuperando curva 409
recuperando curva 410
recuperando curva 411
recuperando curva 412
recuperando curva 413
recuperando curva 414
recuperando curva 415
recuperando curva 416
recuperando curva 417
recuperando curva 418
recuperando curva 419
recuperando curva 420
recuperando curva 421
recuperando curva 422
recuperando curva 423
recuperand

recuperando curva 752
recuperando curva 753
recuperando curva 754
recuperando curva 755
recuperando curva 756
recuperando curva 757
recuperando curva 758
recuperando curva 759
recuperando curva 760
recuperando curva 761
recuperando curva 762
recuperando curva 763
recuperando curva 764
recuperando curva 765
recuperando curva 766
recuperando curva 767
recuperando curva 768
recuperando curva 769
recuperando curva 770
recuperando curva 771
recuperando curva 772
recuperando curva 773
recuperando curva 774
recuperando curva 775
recuperando curva 776
recuperando curva 777
recuperando curva 778
recuperando curva 779
recuperando curva 780
recuperando curva 781
recuperando curva 782
recuperando curva 783
recuperando curva 784
recuperando curva 785
recuperando curva 786
recuperando curva 787
recuperando curva 788
recuperando curva 789
recuperando curva 790
recuperando curva 791
recuperando curva 792
recuperando curva 793
recuperando curva 794
recuperando curva 795
recuperando curva 796
recuperand

recuperando curva 1119
recuperando curva 1120
recuperando curva 1121
recuperando curva 1122
recuperando curva 1123
recuperando curva 1124
recuperando curva 1125
recuperando curva 1126
recuperando curva 1127
recuperando curva 1128
recuperando curva 1129
recuperando curva 1130
recuperando curva 1131
recuperando curva 1132
recuperando curva 1133
recuperando curva 1134
recuperando curva 1135
recuperando curva 1136
recuperando curva 1137
recuperando curva 1138
recuperando curva 1139
recuperando curva 1140
recuperando curva 1141
recuperando curva 1142
recuperando curva 1143
recuperando curva 1144
recuperando curva 1145
recuperando curva 1146
recuperando curva 1147
recuperando curva 1148
recuperando curva 1149
recuperando curva 1150
recuperando curva 1151
recuperando curva 1152
recuperando curva 1153
recuperando curva 1154
recuperando curva 1155
recuperando curva 1156
recuperando curva 1157
recuperando curva 1158
recuperando curva 1159
recuperando curva 1160
recuperando curva 1161
recuperando

recuperando curva 1476
recuperando curva 1477
recuperando curva 1478
recuperando curva 1479
recuperando curva 1480
recuperando curva 1481
recuperando curva 1482
recuperando curva 1483
recuperando curva 1484
recuperando curva 1485
recuperando curva 1486
recuperando curva 1487
recuperando curva 1488
recuperando curva 1489
recuperando curva 1490
recuperando curva 1491
recuperando curva 1492
recuperando curva 1493
recuperando curva 1494
recuperando curva 1495
recuperando curva 1496
recuperando curva 1497
recuperando curva 1498
recuperando curva 1499
recuperando curva 1500
recuperando curva 1501
recuperando curva 1502
recuperando curva 1503
recuperando curva 1504
recuperando curva 1505
recuperando curva 1506
recuperando curva 1507
recuperando curva 1508
recuperando curva 1509
recuperando curva 1510
recuperando curva 1511
recuperando curva 1512
recuperando curva 1513
recuperando curva 1514
recuperando curva 1515
recuperando curva 1516
recuperando curva 1517
recuperando curva 1518
recuperando

...

recuperando curva 6576
recuperando curva 6577
recuperando curva 6578
recuperando curva 6579
recuperando curva 6580
recuperando curva 6581
recuperando curva 6582
recuperando curva 6583
recuperando curva 6584
recuperando curva 6585
recuperando curva 6586
recuperando curva 6587
recuperando curva 6588
recuperando curva 6589
recuperando curva 6590
recuperando curva 6591
recuperando curva 6592
recuperando curva 6593
recuperando curva 6594
recuperando curva 6595
recuperando curva 6596
recuperando curva 6597
recuperando curva 6598
recuperando curva 6599
recuperando curva 6600
recuperando curva 6601
recuperando curva 6602
recuperando curva 6603
recuperando curva 6604
recuperando curva 6605
recuperando curva 6606
recuperando curva 6607
recuperando curva 6608
recuperando curva 6609
recuperando curva 6610
recuperando curva 6611
recuperando curva 6612
recuperando curva 6613
recuperando curva 6614
recuperando curva 6615
recuperando curva 6616
recuperando curva 6617
recuperando curva 6618
recuperando

recuperando curva 6933
recuperando curva 6934
recuperando curva 6935
recuperando curva 6936
recuperando curva 6937
recuperando curva 6938
recuperando curva 6939
recuperando curva 6940
recuperando curva 6941
recuperando curva 6942
recuperando curva 6943
recuperando curva 6944
recuperando curva 6945
recuperando curva 6946
recuperando curva 6947
recuperando curva 6948
recuperando curva 6949
recuperando curva 6950
recuperando curva 6951
recuperando curva 6952
recuperando curva 6953
recuperando curva 6954
recuperando curva 6955
recuperando curva 6956
recuperando curva 6957
recuperando curva 6958
recuperando curva 6959
recuperando curva 6960
recuperando curva 6961
recuperando curva 6962
recuperando curva 6963
recuperando curva 6964
recuperando curva 6965
recuperando curva 6966
recuperando curva 6967
recuperando curva 6968
recuperando curva 6969
recuperando curva 6970
recuperando curva 6971
recuperando curva 6972
recuperando curva 6973
recuperando curva 6974
recuperando curva 6975
recuperando

recuperando curva 7290
recuperando curva 7291
recuperando curva 7292
recuperando curva 7293
recuperando curva 7294
recuperando curva 7295
recuperando curva 7296
recuperando curva 7297
recuperando curva 7298
recuperando curva 7299
recuperando curva 7300
recuperando curva 7301
recuperando curva 7302
recuperando curva 7303
recuperando curva 7304
recuperando curva 7305
recuperando curva 7306
recuperando curva 7307
recuperando curva 7308
recuperando curva 7309
recuperando curva 7310
recuperando curva 7311
recuperando curva 7312
recuperando curva 7313
recuperando curva 7314
recuperando curva 7315
recuperando curva 7316
recuperando curva 7317
recuperando curva 7318
recuperando curva 7319
recuperando curva 7320
recuperando curva 7321
recuperando curva 7322
recuperando curva 7323
recuperando curva 7324
recuperando curva 7325
recuperando curva 7326
recuperando curva 7327
recuperando curva 7328
recuperando curva 7329
recuperando curva 7330
recuperando curva 7331
recuperando curva 7332
recuperando

recuperando curva 7647
recuperando curva 7648
recuperando curva 7649
recuperando curva 7650
recuperando curva 7651
recuperando curva 7652
recuperando curva 7653
recuperando curva 7654
recuperando curva 7655
recuperando curva 7656
recuperando curva 7657
recuperando curva 7658
recuperando curva 7659
recuperando curva 7660
recuperando curva 7661
recuperando curva 7662
recuperando curva 7663
recuperando curva 7664
recuperando curva 7665
recuperando curva 7666
recuperando curva 7667
recuperando curva 7668
recuperando curva 7669
recuperando curva 7670
recuperando curva 7671
recuperando curva 7672
recuperando curva 7673
recuperando curva 7674
recuperando curva 7675
recuperando curva 7676
recuperando curva 7677
recuperando curva 7678
recuperando curva 7679
recuperando curva 7680
recuperando curva 7681
recuperando curva 7682
recuperando curva 7683
recuperando curva 7684
recuperando curva 7685
recuperando curva 7686
recuperando curva 7687
recuperando curva 7688
recuperando curva 7689
recuperando

recuperando curva 8004
recuperando curva 8005
recuperando curva 8006
recuperando curva 8007
recuperando curva 8008
recuperando curva 8009
recuperando curva 8010
recuperando curva 8011
recuperando curva 8012
recuperando curva 8013
recuperando curva 8014
recuperando curva 8015
recuperando curva 8016
recuperando curva 8017
recuperando curva 8018
recuperando curva 8019
recuperando curva 8020
recuperando curva 8021
recuperando curva 8022
recuperando curva 8023
recuperando curva 8024
recuperando curva 8025
recuperando curva 8026
recuperando curva 8027
recuperando curva 8028
recuperando curva 8029
recuperando curva 8030
recuperando curva 8031
recuperando curva 8032
recuperando curva 8033
recuperando curva 8034
recuperando curva 8035
recuperando curva 8036
recuperando curva 8037
recuperando curva 8038
recuperando curva 8039
recuperando curva 8040
recuperando curva 8041
recuperando curva 8042
recuperando curva 8043
recuperando curva 8044
recuperando curva 8045
recuperando curva 8046
recuperando

In [ ]:
final_npy = np.asarray(final_npy)
np.save('/work/work_teamEXOPLANET/MTF_gabo/npys/MTF_time_u10_d10.npy', final_npy)

In [ ]:
!ls 